In [1]:
import psycopg2 as pg2
import pandas as pd

con = pg2.connect(host='this_postgres', user='postgres', database='postgres')
con.autocommit = True
cur = con.cursor()

def select(sql):
    return pd.read_sql(sql,con)

In [2]:
sql = '''
drop table if exists sales_month;
create table sales_month as
select 

t.*,
s.invoice_cnt,
s.unitprice_sum,
1 as customer,
case when t.reg_month = t.month then 1 else 0 end as new,
case when s.invoice_cnt > 0 then 1 else 0 end as active,


extract(year from age(t.month, max(case when s.invoice_cnt > 0 then t.month else null end) 
over (partition by t.customerid order by t.month))) * 12 +
extract(month from age(t.month, max(case when s.invoice_cnt > 0 then t.month else null end) 
over (partition by t.customerid order by t.month))) as last_order_cnt,


extract(year from age(t.month, t.reg_month)) * 12 +
extract(month from age(t.month, t.reg_month)) as lifetime






from user_month_1 t 

left join sales_month_agg s on t.customerid = s.customerid and
t.month = s.month

'''

In [3]:
cur.execute(sql)

In [4]:
sql = '''
select 
t.customerid, t.month, 
t.target,
max(s.month),

extract(year from age(t.month, max(s.month))) * 12 +
extract(month from age(t.month, max(s.month))) as last_order_month,


sum(s.invoice_cnt) as invoice_cnt,

sum(s.unitprice_sum) as unitprice_sum



from 

(select t.customerid, t.month, 
case when t.invoice_cnt > 0 then 1 else 0 end as target

from sales_month t
where t.month = '2011-10-1' and t.lifetime > 0) t


left join sales_month_agg s on t.customerid = s.customerid
and s.month < t.month


group by t.customerid, t.month, 
t.target
'''

In [5]:
df = select(sql)

In [6]:
df

,customerid,month,target,max,last_order_month,invoice_cnt,unitprice_sum
0,12346.0,2011-10-01,0,2011-01-01,9.0,2.0,2.08
1,12347.0,2011-10-01,1,2011-08-01,2.0,5.0,346.91
2,12348.0,2011-10-01,0,2011-09-01,1.0,4.0,178.71
3,12350.0,2011-10-01,0,2011-02-01,8.0,1.0,65.30
4,12352.0,2011-10-01,0,2011-09-01,1.0,10.0,2124.57
...,...,...,...,...,...,...,...
3653,18280.0,2011-10-01,0,2011-03-01,7.0,1.0,47.65
3654,18281.0,2011-10-01,0,2011-06-01,4.0,1.0,39.36
3655,18282.0,2011-10-01,0,2011-08-01,2.0,2.0,39.16
3656,18283.0,2011-10-01,1,2011-09-01,1.0,10.0,753.31


https://stackoverflow.com/questions/50804120/how-do-i-get-the-percentile-for-a-row-in-a-pandas-dataframe

In [7]:
df['recency'] = df['last_order_month'].rank(pct=True,ascending=False,method='max')

In [8]:
df

,customerid,month,target,max,last_order_month,invoice_cnt,unitprice_sum,recency
0,12346.0,2011-10-01,0,2011-01-01,9.0,2.0,2.08,0.088026
1,12347.0,2011-10-01,1,2011-08-01,2.0,5.0,346.91,0.644068
2,12348.0,2011-10-01,0,2011-09-01,1.0,4.0,178.71,1.000000
3,12350.0,2011-10-01,0,2011-02-01,8.0,1.0,65.30,0.126845
4,12352.0,2011-10-01,0,2011-09-01,1.0,10.0,2124.57,1.000000
...,...,...,...,...,...,...,...,...
3653,18280.0,2011-10-01,0,2011-03-01,7.0,1.0,47.65,0.186987
3654,18281.0,2011-10-01,0,2011-06-01,4.0,1.0,39.36,0.399672
3655,18282.0,2011-10-01,0,2011-08-01,2.0,2.0,39.16,0.644068
3656,18283.0,2011-10-01,1,2011-09-01,1.0,10.0,753.31,1.000000


In [9]:
len(df[df['last_order_month'] >= 9]) / len(df)

0.08802624384909787

In [10]:
df['frequency'] = df['invoice_cnt'].rank(pct=True,method='max')

In [11]:
df['monetary'] = df['unitprice_sum'].rank(pct=True,method='max')

In [12]:
df

,customerid,month,target,max,last_order_month,invoice_cnt,unitprice_sum,recency,frequency,monetary
0,12346.0,2011-10-01,0,2011-01-01,9.0,2.0,2.08,0.088026,0.536359,0.012028
1,12347.0,2011-10-01,1,2011-08-01,2.0,5.0,346.91,0.644068,0.793876,0.822307
2,12348.0,2011-10-01,0,2011-09-01,1.0,4.0,178.71,1.000000,0.739475,0.648715
3,12350.0,2011-10-01,0,2011-02-01,8.0,1.0,65.30,0.126845,0.345817,0.333789
4,12352.0,2011-10-01,0,2011-09-01,1.0,10.0,2124.57,1.000000,0.920995,0.991799
...,...,...,...,...,...,...,...,...,...,...
3653,18280.0,2011-10-01,0,2011-03-01,7.0,1.0,47.65,0.186987,0.345817,0.253964
3654,18281.0,2011-10-01,0,2011-06-01,4.0,1.0,39.36,0.399672,0.345817,0.211044
3655,18282.0,2011-10-01,0,2011-08-01,2.0,2.0,39.16,0.644068,0.536359,0.209677
3656,18283.0,2011-10-01,1,2011-09-01,1.0,10.0,753.31,1.000000,0.920995,0.937124


In [13]:
df['score'] = (df['recency']  + df['frequency']  + df['monetary'] ) / 3

In [14]:
df

,customerid,month,target,max,last_order_month,invoice_cnt,unitprice_sum,recency,frequency,monetary,score
0,12346.0,2011-10-01,0,2011-01-01,9.0,2.0,2.08,0.088026,0.536359,0.012028,0.212138
1,12347.0,2011-10-01,1,2011-08-01,2.0,5.0,346.91,0.644068,0.793876,0.822307,0.753417
2,12348.0,2011-10-01,0,2011-09-01,1.0,4.0,178.71,1.000000,0.739475,0.648715,0.796063
3,12350.0,2011-10-01,0,2011-02-01,8.0,1.0,65.30,0.126845,0.345817,0.333789,0.268817
4,12352.0,2011-10-01,0,2011-09-01,1.0,10.0,2124.57,1.000000,0.920995,0.991799,0.970931
...,...,...,...,...,...,...,...,...,...,...,...
3653,18280.0,2011-10-01,0,2011-03-01,7.0,1.0,47.65,0.186987,0.345817,0.253964,0.262256
3654,18281.0,2011-10-01,0,2011-06-01,4.0,1.0,39.36,0.399672,0.345817,0.211044,0.318845
3655,18282.0,2011-10-01,0,2011-08-01,2.0,2.0,39.16,0.644068,0.536359,0.209677,0.463368
3656,18283.0,2011-10-01,1,2011-09-01,1.0,10.0,753.31,1.000000,0.920995,0.937124,0.952706


In [15]:
df['target'].mean()

0.29332968835429196

In [16]:
df[df['score'] >= .7]['target'].mean()

0.4969108561341571

In [17]:
df['score_pct'] = df['score'].rank(pct=True,ascending=False,method='max')

In [18]:
import numpy as np

In [19]:
np.linspace(0,1,101)

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99, 1.  ])

In [20]:
df['score_cut'] = pd.cut(df['score_pct'],np.linspace(0,1,101))

In [21]:
lift = df.groupby('score_cut')['target'].agg(['sum','count']).reset_index()

In [22]:
lift

,score_cut,sum,count
0,"(0.0, 0.01]",35,36
1,"(0.01, 0.02]",32,37
2,"(0.02, 0.03]",27,36
3,"(0.03, 0.04]",24,37
4,"(0.04, 0.05]",27,36
...,...,...,...
95,"(0.95, 0.96]",4,36
96,"(0.96, 0.97]",4,36
97,"(0.97, 0.98]",3,37
98,"(0.98, 0.99]",5,36


In [23]:
lift['target_cum'] = lift['sum'].cumsum()

In [24]:
lift['count_cum'] = lift['count'].cumsum()

In [25]:
lift['target_pct'] = lift['target_cum'] / lift['sum'].sum()

In [26]:
lift['score_pct'] = lift['score_cut'].apply(lambda x: x.right).astype(float)

In [27]:
lift

,score_cut,sum,count,target_cum,count_cum,target_pct,score_pct
0,"(0.0, 0.01]",35,36,35,36,0.032619,0.01
1,"(0.01, 0.02]",32,37,67,73,0.062442,0.02
2,"(0.02, 0.03]",27,36,94,109,0.087605,0.03
3,"(0.03, 0.04]",24,37,118,146,0.109972,0.04
4,"(0.04, 0.05]",27,36,145,182,0.135135,0.05
...,...,...,...,...,...,...,...
95,"(0.95, 0.96]",4,36,1058,3511,0.986021,0.96
96,"(0.96, 0.97]",4,36,1062,3547,0.989748,0.97
97,"(0.97, 0.98]",3,37,1065,3584,0.992544,0.98
98,"(0.98, 0.99]",5,36,1070,3620,0.997204,0.99


In [28]:
lift['uplift'] = lift['target_pct'] / lift['score_pct']

In [29]:
lift[lift['score_cut'].isin([.05,.1,.15,.2,.3,.4,.5])]

,score_cut,sum,count,target_cum,count_cum,target_pct,score_pct,uplift
4,"(0.04, 0.05]",27,36,145,182,0.135135,0.05,2.702703
9,"(0.09, 0.1]",14,37,241,365,0.224604,0.10,2.246039
14,"(0.14, 0.15]",17,36,318,548,0.296365,0.15,1.975769
19,"(0.19, 0.2]",11,36,409,731,0.381174,0.20,1.905871
29,"(0.29, 0.3]",13,37,549,1097,0.511650,0.30,1.705499
39,"(0.39, 0.4]",11,36,678,1462,0.631873,0.40,1.579683
49,"(0.49, 0.5]",13,37,786,1829,0.732526,0.50,1.465051
